In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
import matplotlib.pyplot as plt


In [2]:
trn = r'/content/drive/MyDrive/Colab Notebooks/MRNet-v1.0/train'
vld = r'/content/drive/MyDrive/Colab Notebooks/MRNet-v1.0/valid'
trn_labels_path = r'/content/drive/MyDrive/Colab Notebooks/MRNet-v1.0/train-acl.csv'
vldn_labels_path = r'/content/drive/MyDrive/Colab Notebooks/MRNet-v1.0/valid-acl.csv'

In [5]:
a = [file for file in os.listdir(trn+r'/axial')]
c = [file for file in os.listdir(trn+r'/coronal')]
s = [file for file in os.listdir(trn+r'/sagittal')]

a_vldn = [file for file in os.listdir(vld+r'/axial')]
c_vldn = [file for file in os.listdir(vld+r'/coronal')]
s_vldn = [file for file in os.listdir(vld+r'/sagittal')]


In [28]:
acl_img = []

for file in a:
    temp=np.load(os.path.join(trn,'axial',file))
    temp = np.reshape(temp,(256,256,temp.shape[0]))
    assert temp.shape[0]>10

    tmp_shape = temp.shape[2] // 2
    low,high = tmp_shape - 5, tmp_shape + 6
    temp = temp[:, :, low:high]
    x_dim,y_dim,z_dim = temp.shape
    temp= np.reshape(temp,(x_dim,y_dim,z_dim,1))
    acl_img.append(temp)

a_img = np.array(acl_img)
print(np.shape(acl_img))

trn_labels = pd.read_csv(trn_labels_path,header=None)
trn_labels = np.reshape(trn_labels[1].to_numpy(), (1130, 1))


(1130, 256, 256, 11, 1)


In [29]:
acl_img_vldn = []

for file in a_vldn:
    temp=np.load(os.path.join(vld, 'axial', file))
    temp = np.reshape(temp, (256, 256, temp.shape[0]))
    assert temp.shape[0] > 10

    tmp_shape = temp.shape[2] // 2
    low, high = tmp_shape - 5, tmp_shape + 6
    temp = temp[:, :, low:high]
    x_dim, y_dim, z_dim = temp.shape
    temp= np.reshape(temp, (x_dim, y_dim, z_dim, 1))
    acl_img_vldn.append(temp)

acl_img_vldn = np.array(acl_img_vldn)
print(np.shape(acl_img_vldn))
vldn_labels = pd.read_csv(vldn_labels_path, header=None)
vldn_labels = np.reshape(vldn_labels[1].to_numpy(), (len(vldn_labels), 1))


(120, 256, 256, 11, 1)


In [30]:
# model = models.Sequential()
# model.add(layers.Conv2D(16, (3,3), activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=(256, 256, 10)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(32, (3,3), activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=(256, 256, 10)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(64, (3,3), activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=(256, 256, 10)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.BatchNormalization())

# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
# model.add(layers.Dense(2, activation = 'softmax'))
# model.summary()

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv3D(1, (3, 3, 3), input_shape=(256, 256, 11, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv3D(1, (3, 3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv3D(1, (3, 3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv3D(1, (3, 3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Reshape((248,248,3)),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Reshape((224,224,3))
 
])

base_model =tf.keras.applications.resnet50.ResNet50(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000
)

base_model_conv = tf.keras.Model(inputs = base_model.input, outputs = base_model.layers[-2].output)

model.add(base_model_conv)
model.add(layers.Dense(1000, activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
model.add(layers.Dense(500, activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 254, 254, 9, 1)    28        
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 9, 1)   4         
 ormalization)                                                   
                                                                 
 conv3d_1 (Conv3D)           (None, 252, 252, 7, 1)    28        
                                                                 
 batch_normalization_1 (Batc  (None, 252, 252, 7, 1)   4         
 hNormalization)                                                 
                                                                 
 conv3d_2 (Conv3D)           (None, 250, 250, 5, 1)    28        
                                                                 
 batch_normalization_2 (Batc  (None, 250, 250, 5, 1)   4

In [32]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, name='Adam'),
              loss = losses.BinaryCrossentropy(from_logits = False),
              metrics=[
                  tf.keras.metrics.Accuracy(name='accuracy', dtype=None),
                  tf.keras.metrics.AUC()
              ]
)
                  

In [33]:
hist = model.fit(acl_img,
                trn_labels,
                validation_data=(acl_img_vldn, vldn_labels),
                epochs=25)

Epoch 1/25
36/36 [==============================] - 68s 2s/step loss: 2.0269 - accuracy: 0.2707 - val_loss: 1.3401 - val_accuracy: 0.5000
Epoch 2/25
36/36 [==============================] - 56s 2s/step - loss: 1.2022 - accuracy: 0.5862 - val_loss: 0.8101 - val_accuracy: 0.7500
Epoch 3/25
36/36 [==============================] - 56s 2s/step - loss: 0.8603 - accuracy: 0.7155 - val_loss: 0.6770 - val_accuracy: 0.7500
Epoch 4/25
36/36 [==============================] - 56s 2s/step - loss: 0.6491 - accuracy: 0.7810 - val_loss: 1.1773 - val_accuracy: 0.6000
Epoch 5/25
36/36 [==============================] - 56s 2s/step loss: 0.5203 - accuracy: 0.8397 - val_loss: 0.3212 - val_accuracy: 0.8500
Epoch 6/25
36/36 [==============================] - 56s 2s/step loss: 0.4610 - accuracy: 0.8569 - val_loss: 0.6156 - val_accuracy: 0.8500
Epoch 7/25
36/36 [==============================] - 56s 2s/step loss: 0.4227 - accuracy: 0.8595 - val_loss: 0.8312 - val_accuracy: 0.7000
Epoch 8/25
36/36 [==========